In [ ]:
import numpy as np
import torch.nn as nn
from dataset import CityscapesDataset
from torchvision.datasets import Cityscapes
import torch.nn.functional as F
from GenISP import *
from unet import *
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import os 
from tqdm import trange

In [ ]:
# ALL CREDITS TO https://github.com/goldbattle/pytorch_unet/blob/master/script_train.py FOR DATASET CLASS!
# path = "/Users/charlesdowns/Desktop/cityscapes_mini/" # Mini dataset on Macbook
path = "D:\Deep learing low light images dataset\cityscapes_mini\cityscapes_mini"
img_data = CityscapesDataset(path, split='train', mode='fine')
img_batch = torch.utils.data.DataLoader(img_data, batch_size=1, shuffle=True, num_workers=16)

In [ ]:
# Hyperparameters and stuff
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.backends.mps.is_available():
    device = torch.device("mps")
print(device)

# Set model parameters
epochs = 5
learning_rate = 0.001
in_c = 3

# Create the model
in_channels = 3 # Raw RGB image
hidden_channels = [16, 32, 128, 16, 64, 3]
out_features = 3 # GenISP output channels
PreNet = GenISPV2( in_channels, hidden_channels, out_features)
UNet = UNet(in_c, img_data.num_classes, 64)
# Net = nn.Sequential(PreNet,UNet)
PreNet.to(device)
UNet.to(device)
train_loss = []

# Since our goal here is to segment, we will use the cross entropy loss function and 4 classes (background, road, sky, car)
criterion = nn.CrossEntropyLoss() # for segment loss. could also be reconstruction loss, then it is N1 loss 
num_classes = img_data.num_classes
print(num_classes)
optimizer = torch.optim.Adam(UNet.parameters(), lr=learning_rate)

In [ ]:
t = trange(epochs, desc="Epochs", leave=True)
for epoch in t:
    # TODO: use tqdm for progress bar
    for idx_batch, (imagergb, labelmask, labelrgb) in enumerate(img_batch):
        imagergb = imagergb.to(device)
        labelmask = labelmask.to(device)
        labelrgb = labelrgb.to(device)
        # print(imagergb.shape)
        optimizer.zero_grad()
        output1 = PreNet(imagergb)
        output2 = F.pad(input=output1, pad=(2, 2, 2, 2))
        output = UNet(output2)
        loss = criterion(output, labelmask)
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())

        if idx_batch % 1000 == 0:
            print("Epoch: {}/{}, Batch: {}/{}, Loss: {:.4f}".format(epoch+1, epochs, idx_batch+1, len(img_batch), loss.item()))
            # Display the output and ground truth next to eachother 
            output = output.detach().cpu().numpy()
            output = np.argmax(output, axis=1)
            output = output.reshape((output.shape[1], output.shape[2]))
            labelmask = labelmask.detach().cpu().numpy()
            labelmask = labelmask.reshape((labelmask.shape[1], labelmask.shape[2]))
            # plt.imshow(output)
            # plt.show()
            # plt.imshow(labelmask)
            # plt.show()
            
            
            # concat output and labelmask
            output = np.concatenate((output, labelmask), axis=1)
            plt.imshow(output)
            plt.show()


In [ ]:
# Plot the training loss
plt.plot(train_loss)
plt.title("Train Loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.show()

# Save the model
torch.save(Net.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")